# Topic Model for POTUS Speech Corpus using Gensim

### Imports

In [55]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamLi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Import stopwords from NLTK

In [57]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Import corpus

In [58]:
COLUMNS=['doc_id','date','pres','title','speech']
df = pd.DataFrame(columns=COLUMNS)
PRES = 'obama'
pres_list = "adams arthur bharrison buchanan bush carter cleveland clinton coolidge eisenhower fdroosevelt fillmore ford garfield grant gwbush harding harrison hayes hoover jackson jefferson johnson jqadams kennedy lbjohnson lincoln madison mckinley monroe nixon obama pierce polk reagan roosevelt taft taylor truman trump tyler vanburen washington wilson".split()

In [59]:
import os
import re
_id = 1
for filename in os.listdir('./speeches'):
    if filename == '.DS_Store':
        continue
    for speech in os.listdir('./speeches/' + filename):
        temp = open('./speeches/' + filename + '/' + speech, 'r', encoding='utf-8').readlines()
        obj = {}
        obj['doc_id'] = _id
        date = re.findall('"([^"]*)"', temp[1])
        obj['date'] = date[0] if len(date) > 0 else None
        obj['pres'] = filename
        obj['title'] = re.findall('"([^"]*)"', temp[0])[0]
        obj['speech']= "".join(temp[2:])
    
        obj = pd.DataFrame(obj, index=[0])
        df = df.append(obj, ignore_index=True)
        _id += 1
df = df.set_index("doc_id")
df.head()

,date,pres,title,speech
doc_id,,,,
1,"August 10, 1927",coolidge,Address at the Opening of Work on Mount Rushmo...,We have come here to dedicate a cornerstone th...
2,"December 8, 1925",coolidge,Third Annual Message,Members of the Congress: In meeting the consti...
3,"December 6, 1923",coolidge,First Annual Message,Since the close of the last Congress the Natio...
4,"October 20, 1925",coolidge,Message Regarding Relationship of Church and S...,"Mr. Moderator, Members Of The Council:\nIt is ..."
5,"March 4, 1925",coolidge,Inaugural Address,\nMy Countrymen:\n\nNo one can contemplate cur...


### Adding extra columns

In [60]:
df[['month', 'day', 'year']] = pd.DataFrame([ str(x).split() for x in df['date'].tolist() ])
months = ['', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 
              'August', 'September', 'October', 'November', 'December']
df['month'] = pd.Series([ str(int(months.index(str(x)))) if str(x) in months else 0 for x in df['month'].tolist()])
df['day'] = pd.Series([str(x)[0:len(str(x))-1] if x is not None else 0 for x in df['day'].tolist() ])
df.head()

,date,pres,title,speech,month,day,year
doc_id,,,,,,,
1,"August 10, 1927",coolidge,Address at the Opening of Work on Mount Rushmo...,We have come here to dedicate a cornerstone th...,12,6,1925
2,"December 8, 1925",coolidge,Third Annual Message,Members of the Congress: In meeting the consti...,10,20,1923
3,"December 6, 1923",coolidge,First Annual Message,Since the close of the last Congress the Natio...,3,4,1925
4,"October 20, 1925",coolidge,Message Regarding Relationship of Church and S...,"Mr. Moderator, Members Of The Council:\nIt is ...",7,3,1925
5,"March 4, 1925",coolidge,Inaugural Address,\nMy Countrymen:\n\nNo one can contemplate cur...,2,22,1925


### Filter for a specific president

In [61]:
docs = df[df['pres'] == PRES]
docs.head()

,date,pres,title,speech,month,day,year
doc_id,,,,,,,
588,"May 15, 2016",obama,Commencement Address at Rutgers University,Hello Rutgers! <Applause.> R-U rah-rah! <Appla...,8,28,2016
589,"January 12, 2016",obama,2016 State of the Union Address,"Mr. Speaker, Mr. Vice President, Members of Co...",8,31,2008
590,"August 28, 2008",obama,Acceptance Speech at the Democratic National C...,To Chairman Dean and my great friend Dick Durb...,6,15,2010
591,"August 31, 2010",obama,Address on the End of the Combat Mission in Iraq,"Good evening. Tonight, I'd like to talk to you...",1,29,2010
592,"June 15, 2010",obama,Speech on the BP Oil Spill,"Good evening. As we speak, our nation faces a ...",3,15,2009


### Remove bad characters

In [62]:
data = docs.speech.values.tolist()
data = [re.sub('\n+', ' ', sent) for sent in data]
data = [re.sub(r"\<.*\>"," ",sent) for sent in data]

### Tokenize

In [63]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
data_words = list(sent_to_words(data))

print(data_words[:1])

[['hello', 'rutgers']]


### Build the bigram and trigram models

In [64]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['hello', 'rutgers']


### Remove Stopwords, Make Bigrams and Lemmatize

In [15]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['rutger']]


### Create the Dictionary and Corpus needed for Topic Modeling

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('achieve', 4),
  ('affection', 1),
  ('afghanistan', 1),
  ('ago', 1),
  ('agree', 1),
  ('ahead', 3),
  ('alive', 1),
  ('alliance', 1),
  ('alone', 1),
  ('already', 1),
  ('always', 2),
  ('america', 14),
  ('american', 8),
  ('ann', 2),
  ('answer', 5),
  ('anyone', 1),
  ('anything', 1),
  ('apathy', 1),
  ('arc', 1),
  ('arm', 1),
  ('asian', 1),
  ('ask', 2),
  ('asleep', 1),
  ('assemble', 1),
  ('atlanta', 2),
  ('autumn', 1),
  ('awake', 1),
  ('axelrod', 1),
  ('back', 4),
  ('backyard', 1),
  ('bad', 1),
  ('ballot', 2),
  ('banner', 1),
  ('beacon', 1),
  ('bear', 1),
  ('begin', 3),
  ('believe', 1),
  ('belong', 2),
  ('bend', 1),
  ('berlin', 1),
  ('biden', 1),
  ('bill', 1),
  ('birmingham', 1),
  ('bitter', 1),
  ('black', 1),
  ('block', 2),
  ('blue', 1),
  ('bomb', 1),
  ('bond', 1),
  ('bowl', 1),
  ('brave', 3),
  ('break', 1),
  ('breathe', 1),
  ('brick', 2),
  ('bridge', 1),
  ('bright', 1),
  ('bring', 1),
  ('build', 2),
  ('burn', 1),
  ('bus', 1),
  ('

### Build LDA model
Save the model in `./models/`

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.save('./models/'+PRES+'-model')

### Print the Keywords in the topics
Save them to a txt in `./outputs/`

In [103]:
file = open("./output/"+PRES+".txt","w")
for topic in lda_model.print_topics():
    file.write(str(topic[0]) + ": " + topic[1])
file.close()

doc_lda = lda_model[corpus]

### Compute Model Perplexity and Coherence Score

In [45]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.456062006400461

Coherence Score:  0.3547375540499792


In [65]:
def isfloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

gov_filter = set(['year','know','take','go','say','must','today','new', 'good','year','want','need','government','people','country','great', 'united_state','country','man','nation','law', 'america','american','time','tonight','make','help','may','would','congress','thereof','care','rule','place','bill','duty','amount'])
def perPresidentAnalysis(PRES):
    docs = df[df['pres'] == PRES]
    if 0 in docs.shape: return
    data = docs.speech.values.tolist()
    data = [re.sub('\n+', ' ', sent) for sent in data]
    data = [re.sub(r"\<.*\>"," ",sent) for sent in data]
    data_words = list(sent_to_words(data))
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)
    
    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)
    
    # Create Corpus
    texts = data_lemmatized
    
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    lda_model.save('./models/'+PRES+'-model')
    file = open("./output/"+PRES+".txt","w")
    for topic in lda_model.print_topics():
        topics = re.compile('[0-9 .+*"]*').split(topic[1])
#         max_weight=-1
#         max_topic=""
#         for i,word, in enumerate(topics):
#             if isfloat(word):
#                 if max_weight < float(word) and word not in seen and word not in gov_filter:
#                     max_weight = float(word)
#                     max_topic = topics[i+1]
#         seen.add(max_topic)
#         file.write(max_topic +" ")
        for word in topics:
            if word not in gov_filter:
                file.write(word + " ")
    file.close()
    doc_lda = lda_model[corpus]

In [66]:
preses = list(df.pres.unique())
# print(preses)
for president in preses:
    print('Analyzing President',preses.index(president)+1,'of',len(preses),'---',end=" ")
    perPresidentAnalysis(president)
    print('Done')

Analyzing President 1 of 45 --- Done
Analyzing President 2 of 45 --- Done
Analyzing President 3 of 45 --- Done
Analyzing President 4 of 45 --- Done
Analyzing President 5 of 45 --- Done
Analyzing President 6 of 45 --- Done
Analyzing President 7 of 45 --- Done
Analyzing President 8 of 45 --- Done
Analyzing President 9 of 45 --- Done
Analyzing President 10 of 45 --- Done
Analyzing President 11 of 45 --- Done
Analyzing President 12 of 45 --- Done
Analyzing President 13 of 45 --- Done
Analyzing President 14 of 45 --- Done
Analyzing President 15 of 45 --- Done
Analyzing President 16 of 45 --- Done
Analyzing President 17 of 45 --- Done
Analyzing President 18 of 45 --- Done
Analyzing President 19 of 45 --- Done
Analyzing President 20 of 45 --- Done
Analyzing President 21 of 45 --- Done
Analyzing President 22 of 45 --- Done
Analyzing President 23 of 45 --- Done
Analyzing President 24 of 45 --- Done
Analyzing President 25 of 45 --- Done
Analyzing President 26 of 45 --- Done
Analyzing President 2

### Visualize the topics

In [66]:
#pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.018859 -0.049189       1        1  50.992073
1      0.084270  0.030464       2        1  27.396254
2     -0.025640 -0.107436       3        1  11.169715
0      0.049418  0.065881       4        1   5.923071
3     -0.126906  0.060280       5        1   4.518895, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
1603  Default   54.000000    insurance   54.000000  30.0000  30.0000
211   Default   81.000000         iraq   81.000000  29.0000  29.0000
422   Default   46.000000        think   46.000000  28.0000  28.0000
171   Default  106.000000          get  106.000000  27.0000  27.0000
272   Default  113.000000       nation  113.000000  26.0000  26.0000
420   Default   50.000000        thank   50.000000  25.0000  25.0000
655   Default   50.000000  